In [15]:
import pandas as pd

# Load pickle data into a DataFrame
df = pd.read_pickle('../_3_data_preprocessing/output/serialized_review_dataframe.pkl')

# Display the DataFrame
print(df)  #Print the first few rows to verify


                                            review_text  F  BR  AU  FI  IR  A  \
0     {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   0   0   0   0  0   
1     {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   0   0   0   0  0   
2     {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   0   0   0   1  0   
3     {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   1   0   0   0  0   
4     {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   1   0   0   0  0   
...                                                 ... ..  ..  ..  ..  .. ..   
1229  {'input_ids': ((tf.Tensor(101, shape=(), dtype...  1   0   0   0   0  0   
1230  {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   1   0   0   0  0   
1231  {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   0   0   0   1  0   
1232  {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   0   0   0   0  0   
1233  {'input_ids': ((tf.Tensor(101, shape=(), dtype...  0   0   0   0   0  0   

      L  LF  MN  O  PE  SC 

In [16]:
from sklearn.model_selection import train_test_split

# Assume your DataFrame is named df
# Extract features (review_text) and labels (F, BR, AU, FI, IR, A, L, LF, MN, O, PE, SC, SE, US, PO)
features = df['review_text']
labels = df[['F', 'BR', 'AU', 'FI', 'IR', 'A', 'L', 'LF', 'MN', 'O', 'PE', 'SC', 'SE', 'US', 'PO']]

# Split the dataset into training and validation sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Now you have your training and validation sets ready
# train_features and train_labels are for training
# val_features and val_labels are for validation


In [17]:
list(train_features)[0]

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int64, numpy=
 array([[  101,  2307, 10439,  4299,  2134,  1043, 15909,  2818,  6686,
           102]], dtype=int64)>,
 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)>}

In [18]:
train_labels

,F,BR,AU,FI,IR,A,L,LF,MN,O,PE,SC,SE,US,PO
869,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
728,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
803,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
644,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
813,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1095,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
860,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [29]:
train_input = {'input_ids':[obj["input_ids"] for obj in list(train_features)],'attention_mask':[obj["attention_mask"] for obj in list(train_features)]}
max_length = 0
# train_input["input_ids"][1].shape[1]
for a in train_input["attention_mask"]:
    max_length = max(max_length,a.shape[1])
print(max_length)

46


ERROR

In [20]:
import tensorflow as tf
from transformers import TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Example: Pad sequences to the maximum length within each batch
# max_length = 128  # Example maximum sequence length

# padded_input_ids = pad_sequences(train_input['input_ids'], maxlen=max_length, padding='post')
# padded_attention_mask = pad_sequences(train_input['attention_mask'], maxlen=max_length, padding='post')

# input_ids = Input(shape=(max_length,), dtype=tf.int32)
# attention_mask = Input(shape=(max_length,), dtype=tf.int32)

# Load BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Example data
train_features, train_labels = train_features[:100], train_labels[:100]  # Example subset of data

# Tokenize input data
train_input = {'input_ids':[obj["input_ids"] for obj in list(train_features)],'attention_mask':[obj["attention_mask"] for obj in list(train_features)]}

bert_output = bert_model(train_input["input_ids"], attention_mask=train_input["attention_mask"])[0]
output = Dense(15, activation='sigmoid')(bert_output)

model = Model(inputs=[train_input["input_ids"], train_input["attention_mask"]], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x=train_input,
          y=train_labels,
          epochs=5,
          batch_size=32,
          validation_split=0.2)

# Evaluate the model
# Evaluate the model
val_input = {'input_ids':[obj["input_ids"] for obj in list(val_features)],'attention_mask':[obj["attention_mask"] for obj in list(val_features)]}
results = model.evaluate(x={'input_ids': val_input["input_ids"], 'attention_mask': val_input["attention_mask"]},
                         y=val_labels)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

IndexError: Exception encountered when calling layer 'tf_bert_model_6' (type TFBertModel).

list index out of range

Call arguments received by layer 'tf_bert_model_6' (type TFBertModel):
  • input_ids=['tf.Tensor(shape=(1, 10), dtype=int64)', 'tf.Tensor(shape=(1, 15), dtype=int64)', 'tf.Tensor(shape=(1, 8), dtype=int64)', 'tf.Tensor(shape=(1, 20), dtype=int64)', 'tf.Tensor(shape=(1, 9), dtype=int64)', 'tf.Tensor(shape=(1, 13), dtype=int64)', 'tf.Tensor(shape=(1, 14), dtype=int64)', 'tf.Tensor(shape=(1, 13), dtype=int64)', 'tf.Tensor(shape=(1, 7), dtype=int64)', 'tf.Tensor(shape=(1, 10), dtype=int64)', 'tf.Tensor(shape=(1, 11), dtype=int64)', 'tf.Tensor(shape=(1, 9), dtype=int64)', 'tf.Tensor(shape=(1, 10), dtype=int64)', 'tf.Tensor(shape=(1, 10), dtype=int64)', 'tf.Tensor(shape=(1, 6), dtype=int64)', 'tf.Tensor(shape=(1, 8), dtype=int64)', 'tf.Tensor(shape=(1, 4), dtype=int64)', 'tf.Tensor(shape=(1, 17), dtype=int64)', 'tf.Tensor(shape=(1, 18), dtype=int64)', 'tf.Tensor(shape=(1, 10),